In [19]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz 

# Example usage:
start_description = '3rd week after Patch Tuesday'
#start time can only contain start time "," and Day of the week
start_time = '8PM, Saturday'
duration_hours = 4
start_timezone = 'bst'

def get_pytz_timezone(start_timezone):
    # Mapping friendly time zone names to pytz time zone names
    timezone_mapping = {
        'argentina': 'America/Argentina/Buenos_Aires',
        'pst': 'America/Los_Angeles',
        'pdt': 'America/Los_Angeles',
        'pst/pdt': 'America/Los_Angeles',
        'PST/PDT': 'America/Los_Angeles',
        'CET/CEST': 'Europe/Berlin',
        'cet/cest': 'Europe/Berlin',
        'chile': 'America/Santiago',
        'bst/gmt': 'Europe/London',
        'bst': 'Europe/London',
        'BST': 'Europe/London',
        'colombia': 'America/Bogota',
        'Central Time(US)': 'US/Central',
        #add
        # !CASE sensitive issue, BST not parsing - bst parsing
    }

    # Try to find a direct timezone match
    pytz_timezone = timezone_mapping.get(start_timezone.lower())

    print(f"mapped timezone: {pytz_timezone}")

    return pytz_timezone

def get_date_range(start_description, start_time, duration_hours, start_timezone):

    print(f"Start Description: {start_description}")
    print(f"Start Time: {start_time}")
    print(f"Duration (hours): {duration_hours}")
    print(f"Start Timezone: {start_timezone}")

    description_parts = start_description.split()
    
    time_parts = start_time.split(', ')
    specified_time = time_parts[0]
    specified_day = time_parts[1]

    start_time_24h = datetime.strptime(specified_time, "%I%p").strftime("%H:%M")

    current_date = datetime.now()

    # Convert start_timezone to pytz time zone
    start_timezone_pytz = get_pytz_timezone(start_timezone)

    # Finding the next Monday after the first Tuesday (Patch Tuesday)
    first_day_of_month = current_date.replace(day=1)
    first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)
    patch_tuesday = first_tuesday + timedelta(days=6)
    next_monday = patch_tuesday + timedelta(days=(7 - patch_tuesday.weekday()) % 7)
        
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_indices = {day: index for index, day in enumerate(day_names)}
    
    start_datetime = None
    end_datetime = None

    if 'next'.lower() in ' '.join(description_parts).lower():
        
        print("Executing 'Next Month' logic...")
        # Extracting the month
        current_date = current_date.replace(day=1) + relativedelta(months=1)
        
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(start_description.split()[0][:-len("th")])
        
        # Calculating the selected weekday of the next month
        first_day_of_next_month = current_date.replace(day=1)
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_day_of_next_month.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_day_of_next_month + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))


        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
    elif 'after'.lower() in ' '.join(description_parts).lower():

        print("Executing 'after Patch Tuesday' logic...")
        if description_parts[0] == '0':
            start_day_index = day_indices[specified_day.capitalize()] - next_monday.weekday()
        else:
            # Extracting the week offset
            week_offset = int(description_parts[0][:-2])
            # Calculate the starting day index within the desired week after Patch Tuesday
            start_day_index = (day_indices[specified_day.capitalize()] - next_monday.weekday() + 7) % 7
            # Adjust to the specified week within the month
            start_day_index += week_offset * 7
            # Adjust if the start_day_index is negative
            if start_day_index < 0:
                start_day_index += 7
        
        # Constructing start date and time
        start_date = next_monday + timedelta(days=start_day_index)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")

    elif 'of the month'.lower() in ' '.join(description_parts).lower():

        print("Executing 'of the' logic...")
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(description_parts[0][:-len("th")])

        # Finding the current month's first Tuesday
        first_day_of_month = current_date.replace(day=1)
        first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)

        # Finding the desired weekday of the month
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_tuesday.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_tuesday + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")

    elif 'last day'.lower() in ' '.join(description_parts).lower():
        
        print("Executing 'Last Day' logic...")

        start_date = current_date.replace(day=1) + relativedelta(months=1, days=-1)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")
    
    else:
        # Additional cases of weekday logic...
        return "Error: Invalid description"

target_timezone = 'Europe/London'
start_date_formatted, end_date_formatted = get_date_range(start_description, start_time, duration_hours, start_timezone)
if start_date_formatted and end_date_formatted:
    print(f"\nConverted to {start_timezone} Time Zone:")
    print(f"Start Date: {start_date_formatted}")
    print(f"End Date: {end_date_formatted}")



Start Description: 3rd week after Patch Tuesday
Start Time: 8PM, Saturday
Duration (hours): 4
Start Timezone: bst
mapped timezone: Europe/London
Executing 'after Patch Tuesday' logic...

Converted to Europe/London Time Zone:
Start Date: 2024-02-03 20:00:00
End Date: 2024-02-04 00:00:00

Converted to bst Time Zone:
Start Date: 2024-02-03 20:00:00
End Date: 2024-02-04 00:00:00


In [3]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz 

# Example usage:
start_description = '3rd Sunday of the month'
#start time can only contain start time "," and Day of the week
start_time = '12AM, Sunday'
duration_hours = 4
start_timezone = 'pst'

def get_pytz_timezone(start_timezone):
    # Mapping friendly time zone names to pytz time zone names
    timezone_mapping = {
        'argentina': 'America/Argentina/Buenos_Aires',
        'pst': 'America/Los_Angeles',
        'pdt': 'America/Los_Angeles',
        'pst/pdt': 'America/Los_Angeles',
        'PST/PDT': 'America/Los_Angeles',
        'CET/CEST': 'Europe/Berlin',
        'cet/cest': 'Europe/Berlin',
        'chile': 'America/Santiago',
        'bst/gmt': 'Europe/London',
        'bst': 'Europe/London',
        'BST': 'Europe/London',
        'colombia': 'America/Bogota',
        'Central Time(US)': 'US/Central',
        #add
        # !CASE sensitive issue, BST not parsing - bst parsing
    }

    # Try to find a direct timezone match
    pytz_timezone = timezone_mapping.get(start_timezone.lower())

    print(f"mapped timezone: {pytz_timezone}")

    return pytz_timezone

def get_date_range(start_description, start_time, duration_hours, start_timezone):

    print(f"Start Description: {start_description}")
    print(f"Start Time: {start_time}")
    print(f"Duration (hours): {duration_hours}")
    print(f"Start Timezone: {start_timezone}")

    description_parts = start_description.split()
    
    time_parts = start_time.split(', ')
    specified_time = time_parts[0]
    specified_day = time_parts[1]

    start_time_24h = datetime.strptime(specified_time, "%I%p").strftime("%H:%M")

    current_date = datetime.now()

    # Convert start_timezone to pytz time zone
    start_timezone_pytz = get_pytz_timezone(start_timezone)

    # Finding the next Monday after the first Tuesday (Patch Tuesday)
    first_day_of_month = current_date.replace(day=1)
    first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)
    patch_tuesday = first_tuesday + timedelta(days=6)
    next_monday = patch_tuesday + timedelta(days=(7 - patch_tuesday.weekday()) % 7)
        
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_indices = {day: index for index, day in enumerate(day_names)}
    
    start_datetime = None
    end_datetime = None

    if 'next'.lower() in ' '.join(description_parts).lower():
        
        print("Executing 'Next Month' logic...")
        # Extracting the month
        current_date = current_date.replace(day=1) + relativedelta(months=1)
        
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(start_description.split()[0][:-len("th")])
        
        # Calculating the selected weekday of the next month
        first_day_of_next_month = current_date.replace(day=1)
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_day_of_next_month.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_day_of_next_month + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))


        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
    elif 'after'.lower() in ' '.join(description_parts).lower():

        print("Executing 'after Patch Tuesday' logic...")
        if description_parts[0] == '0':
            start_day_index = day_indices[specified_day.capitalize()] - next_monday.weekday()
        else:
            # Extracting the week offset
            week_offset = int(description_parts[0][:-2])
            # Calculate the starting day index within the desired week after Patch Tuesday
            start_day_index = (day_indices[specified_day.capitalize()] - next_monday.weekday() + 7) % 7
            # Adjust to the specified week within the month
            start_day_index += week_offset * 7
            # Adjust if the start_day_index is negative
            if start_day_index < 0:
                start_day_index += 7
        
        # Constructing start date and time
        start_date = next_monday + timedelta(days=start_day_index)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")

    elif 'of the month'.lower() in ' '.join(description_parts).lower():

        print("Executing 'of the' logic...")
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(description_parts[0][:-len("th")])

        # Finding the current month's first Tuesday
        first_day_of_month = current_date.replace(day=1)
        first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)

        # Finding the desired weekday of the month
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_tuesday.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_tuesday + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")

    elif 'last day'.lower() in ' '.join(description_parts).lower():
        
        print("Executing 'Last Day' logic...")

        start_date = current_date.replace(day=1) + relativedelta(months=1, days=-1)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        print(f"\nConverted to {target_timezone} Time Zone:")
        print(f"Start Date: {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M:%S"), end_datetime.strftime("%Y-%m-%d %H:%M:%S")
    
    else:
        # Additional cases of weekday logic...
        return "Error: Invalid description"

target_timezone = 'Europe/London'

start_date_formatted, end_date_formatted = get_date_range(start_description, start_time, duration_hours, start_timezone)

#if start_date_formatted and end_date_formatted:
print(f"\nConverted to {start_timezone} Time Zone:")
print(f"Start Date: {start_date_formatted}")
print(f"End Date: {end_date_formatted}")




Start Description: 3rd Sunday of the month
Start Time: 12AM, Sunday
Duration (hours): 4
Start Timezone: pst
mapped timezone: America/Los_Angeles
Executing 'of the' logic...

Converted to Europe/London Time Zone:
Start Date: 2024-01-21 08:00:00
End Date: 2024-01-21 12:00:00

Converted to pst Time Zone:
Start Date: 2024-01-21 00:00:00
End Date: 2024-01-21 04:00:00
